# Tutorial for Real-Time fMRI Cloud-Based Framework

## Overview
This tutorial walks through an example using our cloud-based software framework for real-time fMRI studies, henceforth **rtcloud framework**. We will begin by starting the projectInterface web server, which will wrap our sample.py script. We will then use a brainiak function to create synthetic DICOM data to simulate data produced from the scanner at 2 TRs. Finally, we will open the web server on a localhost that can be displayed in a cell (although you can also open it on a different tab in your browser).

## Table of Contents

* [Before Running This Notebook](#setting_up)
* 

## Before Running This Notebook <a id='setting_up'></a>

Before you are able to run this notebook, you have to complete the installation instructions found in the accompanying [instructions README](https://github.com/brainiak/brainiak-aperture/blob/master/notebooks/real-time/README_INSTRUCTIONS.md). Also remember that you have to complete the following steps every time before you are able to run this notebook:

1. Activate the conda environment for the rtcloud framework:
```
conda activate rtcloud
```
2. On the command line, create a global variable to the full path for the rtcloud framework repo. You must do this in order to use the functions we have created for the framework. And don't forget the forward slash "/" at the end.
```
export RTCLOUD_PATH=/PATH_TO_RTCLOUD/rt-cloud/
```
Double check that you did this correctly by typing the following command. 
```
ECHO $RTCLOUD_PATH
```
This should print the *full* path to the rtcloud framework folder.

## 

## Start the projectInterface Web Server

In [1]:
### ADD FULL PATH TO RTCLOUD REPO AND NOTEBOOK ###
import os
path_to_rtcloud = os.getenv('RTCLOUD_PATH')
if path_to_rtcloud == None:
    print("Please set RTCLOUD_PATH, see instructions")
    raise ValueError
    
path_to_notebook = os.getcwd()  # check and change notebook path as needed

# the python path to the rtcloud repo is set here (to access rtCommon functions)
import sys
sys.path.append(path_to_rtcloud)

# load important modules
from rtCommon.projectInterface import Web
from rtCommon.structDict import StructDict

# declare the scripts that will be wrapped by the projectInterface and accessed through the webServer
scriptToRun = os.path.join(path_to_notebook, 'sample.py')
initScript = os.path.join(path_to_notebook, 'initialize.py')
finalizeScript = os.path.join(path_to_notebook, 'finalize.py')

# MAYBE ADD THIS TO SETTINGS
numTRs = 200

# set up the config and parameters for this rtcloud tutorial
# NOTE: you can also change these parameters in the Settings tab on the web server
config = StructDict({
    'title' : 'rtCloud Tutorial',
    'sessionId' : '20200101T120000',
    'subjectName' : '001_synthetic',
    'datestr' : '20200101',
    
    'isSynthetic' : True,
    'numSynthetic' : numTRs,
    'imgDir' : '/tmp/notebook-simdata',
    
    'subjectNum' : 101,
    'subjectDay' : 1,
    'sessionNum' : 1,
    'runNum' : [1],
    'scanNum' : [14],

    # Plotting settings
    'plotTitle' : 'Realtime Plot',
    'plotXLabel' : 'Sample #',
    'plotYLabel' : 'Value',
    'plotXRangeLow' : 0,
    'plotXRangeHigh' : numTRs,
    'plotYRangeLow' : -1,
    'plotYRangeHigh' : 1,
    'plotAutoRangeX' : False,
    'plotAutoRangeY' : True,

    # important variables about the dicom files
    'dicomNamePattern' : "rt_{run:03d}.dcm",
    'minExpectedDicomSize' : 200000,
})

params = StructDict({
    'fmriPyScript': scriptToRun,
    'initScript': initScript,
    'finalizeScript': finalizeScript,
    'filesremote': False,
    'port': 8889
})

# start the web server
web = Web()
try:
    web.start(params, config, testMode=True)
    print(true)
except RuntimeError as err:
    # ignore event loop already running error
    if str(err) != 'This event loop is already running':
        raise

Listening on: http://localhost:8889


/Users/paulapbrooks/Desktop/practice/rt-cloud/rtCommon/imageHandling.py:19: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  from nibabel.nicom import dicomreaders


***
**IMPORTANT:**

You can only run this cell to start the projectInterface web server only once or else you will get the following runtime error: 
```
Web Server already running
```
If you want to re-start the web server, you have to first SHUT DOWN the kernel. Instructions for doing this can be found [here](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/execute.html#close-a-notebook-kernel-shut-down).

***

## Start the Synthetic Data Generator

We will be using a BrainIAK function to create synthetic fMRI data. After you run the following cell, you can execute `sim_settings` in a separate cell to take a look at the different parameter settings. For instance, you will find that the synthetic data is being produced at a rate of 2 TRs.

In [ ]:
import threading
import brainiak.utils.fmrisim_real_time_generator as sim

sim_settings = sim.default_settings
sim_settings['save_dicom'] = True
sim_settings['save_realtime'] = True
sim_settings['numTRs'] = numTRs

outdir = '/tmp/notebook-simdata'
if not os.path.exists(outdir):
    os.makedirs(outdir)

# run sim.generate_data(outdir, sim_settings) as a thread
syndata_thread = threading.Thread(name='syndata', 
                                  target=sim.generate_data, 
                                  args=(outdir, sim_settings))
syndata_thread.setDaemon(True)
syndata_thread.start()

***
**IMPORTANT:**

You can only run this cell to generate the synthetic data in "real-time" once unless you delete the data that you have already created. Otherwise, all the data (which is found in the `/tmp/notebook-simdata` folder) will be automatically available when you open the web server, instead of having the File Watcher wait for incoming data one TR at a time.

***

## Open the web server on the localhost (you can also this on a separate browser tab).

The first time you open the web server, either in the cell below or in a separate browser tab, you will be prompted to enter a username and password. In this demonstration, you will use "test" for both.

In [3]:
%%html
<iframe src="http://localhost:8889" width="800" height="600"></iframe>

/Users/paulapbrooks/anaconda3/envs/rtcloud_test/lib/python3.7/site-packages/scipy/stats/stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
